# HW04

```yaml
Course:   DS 5001
Module:   04 Homework
Topic:    NLP and the Pipeline
Author:   Ryan Lipps
Date:     10 February 2023
```

## Setup

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import sys
from textparser import TextParser

In [2]:
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [3]:
source_files = f'{data_home}/gutenberg/eliot-set'
data_prefix = 'eliot-set'

In [4]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [5]:
sys.path.append(local_lib)

In [6]:
clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (507,   rf"^\s*Chapter\s+{roman}\s*$"),
    (145, rf"^[A-Z]+\s?[IVXLCDM]*\.$"),
    (6688,  rf"^\s*Chapter\s+{roman}\.\s*$")
]

In [7]:
source_file_list = sorted(glob(f"{source_files}/*.*"))

In [8]:
source_file_list

['/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_ADAM_BEDE-pg507.txt',
 '/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_MIDDLEMARCH-pg145.txt',
 '/Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_THE_MILL_ON_THE_FLOSS-pg6688.txt']

In [9]:
book_data = []
for source_file_path in source_file_list:
    book_id = int(source_file_path.split('-')[-1].split('.')[0].replace('pg',''))
    book_title = source_file_path.split('/')[-1].split('-')[0].replace('_', ' ')
    book_data.append((book_id, source_file_path, book_title))

## Build LIB frame

In [10]:
LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()
LIB

,source_file_path,raw_title
book_id,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE MIDDLEMARCH
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE ADAM BEDE
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE THE MILL ON THE FLOSS


## Add chapter regex

In [11]:
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))
LIB

,source_file_path,raw_title,chap_regex
book_id,,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE MIDDLEMARCH,^[A-Z]+\s?[IVXLCDM]*\.$
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE ADAM BEDE,^\s*Chapter\s+[IVXLCM]+\s*$
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE THE MILL ON THE FLOSS,^\s*Chapter\s+[IVXLCM]+\.\s*$


## Tokenize Corpus

In [12]:
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].raw_title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS

In [13]:
CORPUS = tokenize_collection(LIB)

Tokenizing 145 ELIOT GEORGE MIDDLEMARCH
Importing  /Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_MIDDLEMARCH-pg145.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^[A-Z]+\s?[IVXLCDM]*\.$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 507 ELIOT GEORGE ADAM BEDE
Importing  /Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_ADAM_BEDE-pg507.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*Chapter\s+[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 6688 ELIOT GEORGE THE MILL ON THE FLOSS
Importing  /Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/eliot-set/ELIOT_GEORGE_THE_MILL_ON_THE_F

In [14]:
CORPUS

pos_tuple  pos  token_str  \
book_id chap_id para_num sent_num token_num                                    
145     1       1        0        0                (Who, WP)   WP        Who   
                                  1              (that, WDT)  WDT       that   
                                  2             (cares, VBZ)  VBZ      cares   
                                  3               (much, RB)   RB       much   
                                  4                 (to, TO)   TO         to   
...                                                      ...  ...        ...   
6688    58      69       0        2              (death, NN)   NN      death   
                                  3              (they, PRP)  PRP       they   
                                  4              (were, VBD)  VBD       were   
                                  5                (not, RB)   RB        not   
                                  6          (divided.”, JJ)   JJ  divided.”   

                                            term_str  
book_id chap_id para_num sent_num token_num           
145     1       1        0        0              who  
                                  1             that  
                                  2            cares  
                                  3             much  
                                  4               to  
...                                              ...  
6688    58      69       0        2            death  
                                  3             they  
                                  4             were  
                                  5              not  
                                  6          divided  

[740713 rows x 4 columns]

## Add columns to LIB

In [15]:
LIB['book_len'] = CORPUS.groupby('book_id').term_str.count()
LIB

,source_file_path,raw_title,chap_regex,book_len
book_id,,,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE MIDDLEMARCH,^[A-Z]+\s?[IVXLCDM]*\.$,317805
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE ADAM BEDE,^\s*Chapter\s+[IVXLCM]+\s*$,215404
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE THE MILL ON THE FLOSS,^\s*Chapter\s+[IVXLCM]+\.\s*$,207461


In [16]:
LIB['n_chaps'] = CORPUS.reset_index()[['book_id', 'chap_id']]\
    .drop_duplicates()\
    .groupby('book_id').chap_id.count()

In [17]:
LIB

,source_file_path,raw_title,chap_regex,book_len,n_chaps
book_id,,,,,
145,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE MIDDLEMARCH,^[A-Z]+\s?[IVXLCDM]*\.$,317805,88
507,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE ADAM BEDE,^\s*Chapter\s+[IVXLCM]+\s*$,215404,55
6688,/Users/ryanlipps/Documents/MSDS/DS5001/data/gu...,ELIOT GEORGE THE MILL ON THE FLOSS,^\s*Chapter\s+[IVXLCM]+\.\s*$,207461,58


## Extract vocab

### Look for weirdness

In [18]:
CORPUS[CORPUS.term_str == '']

pos_tuple  pos token_str  \
book_id chap_id para_num sent_num token_num                              
145     5       46       1        0             (…, NN)   NN         …   
        6       0        0        39           (…, NNP)  NNP         …   
        30      33       5        0            ():, VB)   VB        ):   
        39      55       4        8            (…, NNP)  NNP         …   
        43      9        2        17            (&, CC)   CC         &   
        59      21       3        0            ();, NN)   NN        );   
                53       1        0            ();, IN)   IN        );   
        72      3        1        0           (;”, NNS)  NNS        ;”   
        88      4        0        26            (&, CC)   CC         &   
507     21      11       2        68         ((&), NNP)  NNP       (&)   
6688    7       53       3        26            (&, CC)   CC         &   
        25      2        13       17            (&, CC)   CC         &   
                12       0        27            (&, CC)   CC         &   
        26      6        0        14            (&, CC)   CC         &   
        27      3        3        11            (&, CC)   CC         &   
                         4        41            (&, CC)   CC         &   
        34      115      0        56            (&, CC)   CC         &   
        47      55       0        40            (&, CC)   CC         &   
        52      18       5        0            (),, NN)   NN        ),   
        55      1        5        0            (),, NN)   NN        ),   

                                            term_str  
book_id chap_id para_num sent_num token_num           
145     5       46       1        0                   
        6       0        0        39                  
        30      33       5        0                   
        39      55       4        8                   
        43      9        2        17                  
        59      21       3        0                   
                53       1        0                   
        72      3        1        0                   
        88      4        0        26                  
507     21      11       2        68                  
6688    7       53       3        26                  
        25      2        13       17                  
                12       0        27                  
        26      6        0        14                  
        27      3        3        11                  
                         4        41                  
        34      115      0        56                  
        47      55       0        40                  
        52      18       5        0                   
        55      1        5        0

### Drop the weirdness

In [19]:
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS[CORPUS.term_str == '']

,,,,,pos_tuple,pos,token_str,term_str
book_id,chap_id,para_num,sent_num,token_num,,,,


### Chunk POS to first two letters to make POS group

In [20]:
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS.head()

/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_14452/3884858116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CORPUS['pos_group'] = CORPUS.pos.str[:2]


pos_tuple  pos token_str  \
book_id chap_id para_num sent_num token_num                                
145     1       1        0        0             (Who, WP)   WP       Who   
                                  1           (that, WDT)  WDT      that   
                                  2          (cares, VBZ)  VBZ     cares   
                                  3            (much, RB)   RB      much   
                                  4              (to, TO)   TO        to   

                                            term_str pos_group  
book_id chap_id para_num sent_num token_num                     
145     1       1        0        0              who        WP  
                                  1             that        WD  
                                  2            cares        VB  
                                  3             much        RB  
                                  4               to        TO

In [21]:
CORPUS.tail()

pos_tuple  pos  token_str  \
book_id chap_id para_num sent_num token_num                                    
6688    58      69       0        2              (death, NN)   NN      death   
                                  3              (they, PRP)  PRP       they   
                                  4              (were, VBD)  VBD       were   
                                  5                (not, RB)   RB        not   
                                  6          (divided.”, JJ)   JJ  divided.”   

                                            term_str pos_group  
book_id chap_id para_num sent_num token_num                     
6688    58      69       0        2            death        NN  
                                  3             they        PR  
                                  4             were        VB  
                                  5              not        RB  
                                  6          divided        JJ

### Convert to VOCAB table format

In [22]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()    
VOCAB['p'] = VOCAB.n/VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB.head()

,n,n_chars,p,i
term_str,,,,
1,1,1,0.000001,19.498432
1790,1,4,0.000001,19.498432
1799,2,4,0.000003,18.498432
1801more,1,8,0.000001,19.498432
1807,1,4,0.000001,19.498432


## Annotate VOCAB

### Get max POS

In [23]:
VOCAB['max_pos'] = CORPUS[['term_str', 'pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB.head()

,n,n_chars,p,i,max_pos
term_str,,,,,
1,1,1,0.000001,19.498432,CD
1790,1,4,0.000001,19.498432,CD
1799,2,4,0.000003,18.498432,CD
1801more,1,8,0.000001,19.498432,CD
1807,1,4,0.000001,19.498432,CD


In [24]:
VOCAB['max_pos_group'] = CORPUS[['term_str', 'pos_group']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group
term_str,,,,,,
1,1,1,0.000001,19.498432,CD,CD
1790,1,4,0.000001,19.498432,CD,CD
1799,2,4,0.000003,18.498432,CD,CD
1801more,1,8,0.000001,19.498432,CD,CD
1807,1,4,0.000001,19.498432,CD,CD


### Compute POS ambiguity

In [25]:
VOCAB['n_pos'] = CORPUS[['term_str', 'pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str', 'pos']].value_counts().to_frame('n')\
    .reset_index()\
    .groupby('term_str').pos\
    .apply(lambda x: set(x))
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos
term_str,,,,,,,,
1,1,1,0.000001,19.498432,CD,CD,1,{CD}
1790,1,4,0.000001,19.498432,CD,CD,1,{CD}
1799,2,4,0.000003,18.498432,CD,CD,1,{CD}
1801more,1,8,0.000001,19.498432,CD,CD,1,{CD}
1807,1,4,0.000001,19.498432,CD,CD,1,{CD}


In [26]:
VOCAB.sort_values('n_pos', ascending=False).head()

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos
term_str,,,,,,,,
you,7663,3,0.010346,6.594739,PRP,PR,20,"{VBZ, WP, CC, FW, RBR, VBN, VBP, CD, NNS, IN, ..."
youre,217,5,0.000293,11.736881,NN,NN,18,"{VB, VBZ, NNS, PDT, CC, NN, NNP, JJR, FW, MD, ..."
wont,174,4,0.000235,12.055489,VBP,VB,18,"{VB, VBZ, WP, NNS, WDT, VBG, VBD, JJR, NN, NNP..."
o,1103,1,0.001489,9.391215,IN,NN,18,"{VB, VBZ, NNS, WDT, VBD, JJR, NN, NNP, FW, PRP..."
now,1573,3,0.002124,8.879129,RB,RB,18,"{VB, DT, VBZ, NNS, VBD, CC, NN, NNP, RP, FW, P..."


In [27]:
VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))

In [28]:
VOCAB.sort_values('n_pos', ascending=False).head()

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos,n_pos_group,cat_pos_group
term_str,,,,,,,,,,
you,7663,3,0.010346,6.594739,PRP,PR,20,"{VBZ, WP, CC, FW, RBR, VBN, VBP, CD, NNS, IN, ...",12,"{VB, WP, CC, NN, FW, MD, PR, JJ, IN, CD, RB, RP}"
youre,217,5,0.000293,11.736881,NN,NN,18,"{VB, VBZ, NNS, PDT, CC, NN, NNP, JJR, FW, MD, ...",10,"{VB, PD, CC, NN, FW, MD, JJ, PR, IN, RB}"
wont,174,4,0.000235,12.055489,VBP,VB,18,"{VB, VBZ, WP, NNS, WDT, VBG, VBD, JJR, NN, NNP...",10,"{VB, WP, CC, NN, WR, MD, JJ, WD, IN, RB}"
o,1103,1,0.001489,9.391215,IN,NN,18,"{VB, VBZ, NNS, WDT, VBD, JJR, NN, NNP, FW, PRP...",11,"{VB, NN, FW, MD, JJ, PR, IN, WD, CD, RB, RP}"
now,1573,3,0.002124,8.879129,RB,RB,18,"{VB, DT, VBZ, NNS, VBD, CC, NN, NNP, RP, FW, P...",12,"{VB, DT, PD, CC, NN, FW, MD, JJ, PR, IN, RB, RP}"


## Add stopwords

In [29]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [30]:
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
VOCAB.head()

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos,n_pos_group,cat_pos_group,stop
term_str,,,,,,,,,,,
1,1,1,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0
1790,1,4,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0
1799,2,4,0.000003,18.498432,CD,CD,1,{CD},1,{CD},0
1801more,1,8,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0
1807,1,4,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0


In [31]:
VOCAB[VOCAB.stop == 1].sample(10)

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos,n_pos_group,cat_pos_group,stop
term_str,,,,,,,,,,,
re,24,2,0.000032,14.913470,NN,VB,7,"{VB, VBZ, VBD, NN, NNP, JJ, VBP}",3,"{VB, NN, JJ}",1
were,2305,4,0.003112,8.327881,VBD,VB,13,"{VB, VBZ, NNS, VBD, RP, NNP, NN, WRB, JJ, IN, ...",7,"{VB, NN, WR, JJ, IN, RB, RP}",1
himself,1059,7,0.001430,9.449946,PRP,PR,9,"{VB, VBZ, NNS, VBD, NN, JJ, VBP, PDT, PRP}",5,"{VB, PD, NN, PR, JJ}",1
then,1159,4,0.001565,9.319768,RB,RB,14,"{VB, VBZ, NNS, WDT, VBD, RP, NN, NNP, PDT, JJ,...",8,"{VB, PD, NN, JJ, WD, IN, RB, RP}",1
at,4812,2,0.006497,7.266011,IN,IN,10,"{VB, NNS, NN, NNP, JJ, IN, VBP, RB, VBN, RP}",6,"{VB, NN, JJ, IN, RB, RP}",1
it,7933,2,0.010711,6.544782,PRP,PR,13,"{VB, VBZ, NNS, VBG, PDT, NN, NNP, JJ, IN, VBP,...",7,"{VB, PD, NN, PR, JJ, IN, RB}",1
down,870,4,0.001175,9.733561,RP,RP,10,"{VB, VBZ, NNS, NN, JJ, IN, VBN, RB, VBP, RP}",6,"{VB, NN, JJ, IN, RB, RP}",1
now,1573,3,0.002124,8.879129,RB,RB,18,"{VB, DT, VBZ, NNS, VBD, CC, NN, NNP, RP, FW, P...",12,"{VB, DT, PD, CC, NN, FW, MD, JJ, PR, IN, RB, RP}",1
which,3493,5,0.004716,7.728181,WDT,WD,11,"{VB, NNS, WDT, VBD, NN, NNP, JJ, IN, RBR, RB, ...",6,"{VB, NN, WD, JJ, IN, RB}",1


# DOES IT MAKE SENSE THAT THERE ARE DIFFERENT MAX_POS AND MAX_POS_GROUP? I THINK SO

## Add stems

In [32]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

In [33]:
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,n_pos,cat_pos,n_pos_group,cat_pos_group,stop,stem_porter,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,,,,
1,1,1,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0,1,1,1
1790,1,4,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0,1790,1790,1790
1799,2,4,0.000003,18.498432,CD,CD,1,{CD},1,{CD},0,1799,1799,1799
1801more,1,8,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0,1801more,1801more,1801more
1807,1,4,0.000001,19.498432,CD,CD,1,{CD},1,{CD},0,1807,1807,1807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
œdipus,2,6,0.000003,18.498432,NN,NN,1,{NN},1,{NN},0,œdipu,œdipus,œdip
μέγεθος,1,7,0.000001,19.498432,NNP,NN,1,{NNP},1,{NN},0,μέγεθος,μέγεθος,μέγεθος
τι,1,2,0.000001,19.498432,NNP,NN,1,{NNP},1,{NN},0,τι,τι,τι


## Question 1:
What regular expreWhat regular expression did you use to chunk _Middlemarch_ into chapters?

### Answer 1

In [40]:
LIB[LIB.raw_title == 'ELIOT GEORGE MIDDLEMARCH'].chap_regex.values[0]

'^[A-Z]+\\s?[IVXLCDM]*\\.$'

## Question 2:
What is the title of the book that has the most tokens? 

### Answer 2

In [35]:
LIB.loc[LIB.book_len.idxmax()].raw_title

'ELIOT GEORGE MIDDLEMARCH'

## Question 3:
How many chapter level chunks are there in this novel?

### Answer 3:

In [36]:
LIB.loc[LIB.book_len.idxmax()].n_chaps

88

## Question 4:
Among the three stemming algorithms -- Porter, Lancaster, and Snowball --  which is the most aggressive, in terms of the number of words associated with each stem?

### Answer 4:

In [37]:
print(f"Number of unique Porter stems: {VOCAB.groupby('stem_porter').count().shape[0]}")
print(f"Number of unique Lancaster stems: {VOCAB.groupby('stem_lancaster').count().shape[0]}")
print(f"Number of unique Snowball stems: {VOCAB.groupby('stem_snowball').count().shape[0]}")

Number of unique Porter stems: 17547
Number of unique Lancaster stems: 14619
Number of unique Snowball stems: 17210


**Since Lancaster has the fewest rows after grouping by stems, it is the most aggressive.**

## Question 5:
Using the most aggressive stemmer from the previous question, what is the stem with the most associated terms?

### Answer 5:

In [38]:
VOCAB.stem_lancaster.value_counts().iloc[0:1]

stem_lancaster
cont    34
Name: count, dtype: int64